In [60]:
import smtplib
from email_validator import validate_email, EmailNotValidError  # type: ignore
import dns.resolver  # type: ignore
import sqlite3
import random
import string
import pandas as pd # type: ignore
from openpyxl import load_workbook # type: ignore

In [61]:
# Fonction pour créer la base de données et insérer des données aléatoires
def create_database():
    with sqlite3.connect('email_validation.db') as conn:
        c = conn.cursor()
        # Créer la table
        c.execute('''CREATE TABLE IF NOT EXISTS email_validation
                    (email TEXT, syntax_result TEXT, mx_result TEXT, deliverability_result TEXT)''')
        # Insérer des données de test aléatoires
        for _ in range(10):
            email = ''.join(random.choices(string.ascii_lowercase, k=5)) + '@example.com'
            c.execute("INSERT INTO email_validation (email, syntax_result, mx_result, deliverability_result) VALUES (?, ?, ?, ?)",
                      (email, None, None, None))

In [62]:
# Étape 1 : Valider la syntaxe de l'email

def validate_email_syntax(email):
    try:
        valid = validate_email(email)
        return valid.email
    except EmailNotValidError as e:
        return f"Invalid email syntax: {e}"

In [63]:
# Étape 2 : Vérifier les enregistrements MX du domaine de l'email

def check_mx_record(domain):
    try:
        records = dns.resolver.resolve(domain, 'MX')
        mx_record = records[0].exchange
        return f"MX record found: {mx_record}"
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN, dns.resolver.Timeout) as e:
        return f"No MX record found for domain {domain}: {e}"
    except Exception as e:
        return f"An error occurred while checking MX record for domain {domain}: {e}"

In [64]:
# Étape 3 : Vérifier la délivrabilité de l'email

def verify_email_deliverability(email):
    domain = email.split('@')[1]
    try:
        records = dns.resolver.resolve(domain, 'MX')
        mx_record = records[0].exchange.to_text()

        with smtplib.SMTP(mx_record, 25, timeout=30) as server:
            server.set_debuglevel(1)
            server.ehlo()
            server.starttls()
            server.ehlo()
            server.mail('test@example.com')
            code, message = server.rcpt(email)

            if code == 250:
                return f"Email {email} is deliverable."
            else:
                return f"Email {email} is not deliverable: {message}"
    except smtplib.SMTPConnectError as e:
        return f"SMTP connect error: {e}"
    except smtplib.SMTPServerDisconnected as e:
        return f"SMTP server disconnected: {e}"
    except smtplib.SMTPException as e:
        return f"SMTP error checking deliverability for {email}: {e}"
    except Exception as e:
        return f"Error checking deliverability for {email}: {e}"

In [65]:
# Fonction pour enregistrer les résultats dans un fichier Excel

def store_results_in_csv(deliverable_results, non_deliverable_results):
    deliverable_df = pd.DataFrame(deliverable_results, columns=['email', 'syntax_result', 'mx_result', 'deliverability_result'])
    non_deliverable_df = pd.DataFrame(non_deliverable_results, columns=['email', 'syntax_result', 'mx_result', 'deliverability_result'])

    with pd.ExcelWriter('email_validation_results.xlsx', engine='openpyxl') as writer:
        deliverable_df.to_excel(writer, sheet_name='deliverable', index=False)
        non_deliverable_df.to_excel(writer, sheet_name="Non deliverable", index=False)

    
    # Charger le fichier Excel pour ajuster la largeur des colonnes
    wb = load_workbook('email_validation_results.xlsx')

    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        for column_cells in sheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            sheet.column_dimensions[column_cells[0].column_letter].width = length + 2

    wb.save('email_validation_results.xlsx')

In [66]:
# Fonction principale pour valider un email et stocker les résultats
def validate_email_address(email, deliverable_results, non_deliverable_results):
    syntax_result = validate_email_syntax(email)
    print(f"Syntax validation result: {syntax_result}")

    if "Invalid email syntax" in syntax_result:
        mx_result = None
        deliverability_result = None
        non_deliverable_results.append([email, syntax_result, mx_result, deliverability_result])
    else:
        domain = email.split('@')[1]
        mx_result = check_mx_record(domain)
        print(f"MX record check result: {mx_result}")

        if "No MX record found" in mx_result:
            deliverability_result = None
            non_deliverable_results.append([email, syntax_result, mx_result, deliverability_result])
        else:
            deliverability_result = verify_email_deliverability(email)
            print(f"Deliverability check result: {deliverability_result}")
            if "is deliverable" in deliverability_result:
                deliverable_results.append([email, syntax_result, mx_result, deliverability_result])
            else:
                non_deliverable_results.append([email, syntax_result, mx_result, deliverability_result])

In [67]:
# Créer la base de données avec des données aléatoires
create_database()

# Tester les fonctions avec les emails de la base de données
if __name__ == "__main__":
    deliverable_results = []
    non_deliverable_results = []

    with sqlite3.connect('email_validation.db') as conn:
        c = conn.cursor()
        c.execute("SELECT email FROM email_validation")
        emails = c.fetchall()

        for email in emails:
            validate_email_address(email[0], deliverable_results, non_deliverable_results)

    # Stocker les résultats dans un fichier Excel
    store_results_in_csv(deliverable_results, non_deliverable_results)

Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: simoboudane5@gmail.com
MX record check result: MX record found: alt3.gmail-smtp-in.l.google.com.


send: 'ehlo [192.168.1.24]\r\n'
reply: b'250-mx.google.com at your service, [41.250.0.70]\r\n'
reply: b'250-SIZE 157286400\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'mx.google.com at your service, [41.250.0.70]\nSIZE 157286400\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [192.168.1.24]\r\n'
reply: b'250-mx.google.com at your service, [41.250.0.70]\r\n'
reply: b'250-SIZE 157286400\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'mx.google.com at your service, [41.250.0.70]\nSIZE 157286400\n8BITMIME\nENHANCEDSTATUSCODES\nPIPELINING\nCHUN

Deliverability check result: Email simoboudane5@gmail.com is deliverable.
Syntax validation result: hicham.mouti2@gmail.com
MX record check result: MX record found: alt3.gmail-smtp-in.l.google.com.


send: 'ehlo [192.168.1.24]\r\n'
reply: b'250-mx.google.com at your service, [41.250.0.70]\r\n'
reply: b'250-SIZE 157286400\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'mx.google.com at your service, [41.250.0.70]\nSIZE 157286400\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [192.168.1.24]\r\n'
reply: b'250-mx.google.com at your service, [41.250.0.70]\r\n'
reply: b'250-SIZE 157286400\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'mx.google.com at your service, [41.250.0.70]\nSIZE 157286400\n8BITMIME\nENHANCEDSTATUSCODES\nPIPELINING\nCHUN

Deliverability check result: Email hicham.mouti2@gmail.com is deliverable.
Syntax validation result: mohamedboudane6@gmail.com
MX record check result: MX record found: alt4.gmail-smtp-in.l.google.com.


send: 'ehlo [192.168.1.24]\r\n'
reply: b'250-mx.google.com at your service, [41.250.0.70]\r\n'
reply: b'250-SIZE 157286400\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'mx.google.com at your service, [41.250.0.70]\nSIZE 157286400\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [192.168.1.24]\r\n'
reply: b'250-mx.google.com at your service, [41.250.0.70]\r\n'
reply: b'250-SIZE 157286400\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'mx.google.com at your service, [41.250.0.70]\nSIZE 157286400\n8BITMIME\nENHANCEDSTATUSCODES\nPIPELINING\nCHUN

Deliverability check result: Email mohamedboudane6@gmail.com is deliverable.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invalid email syntax: The domain name example.com does not accept email.
Syntax validation result: Invali